In [12]:
# from fincore.storage.connection import get_connection

# con = get_connection()
# con.execute("DROP TABLE IF EXISTS benchmark_membership")
# con.execute("DROP TABLE IF EXISTS benchmarks")
# con.close()


In [13]:
# from fincore.storage.schema import create_schema
# create_schema()


In [14]:
# import fincore.data.benchmarks as b
# import inspect

# print(inspect.getsource(b.register_benchmarks))


In [15]:
# from fincore.data.benchmarks import register_benchmarks
# register_benchmarks()


In [16]:
# from fincore.data.assets import register_nifty50_assets
# register_nifty50_assets()


In [17]:
# from datetime import date
# from fincore.data.universe import add_asset_to_benchmark

# for asset_id in range(1, 6):
#     add_asset_to_benchmark(
#         benchmark_name="NIFTY_50",
#         asset_id=asset_id,
#         start_date=date(2010, 1, 1),
#     )


In [18]:
# from datetime import date
# from fincore.data.yahoo import ingest_prices_for_asset
# from fincore.storage.connection import get_connection

# con = get_connection(read_only=True)
# assets = con.execute(
#     "SELECT asset_id, ticker FROM assets"
# ).fetchall()
# con.close()

# for asset_id, ticker in assets:
#     ingest_prices_for_asset(
#         asset_id=asset_id,
#         ticker=ticker,
#         start_date=date(2015, 1, 1),
#     )


In [19]:
import pandas as pd
from fincore.storage.connection import get_connection
from fincore.analytics.returns import compute_returns, resample_returns

con = get_connection(read_only=True)

df = con.execute("""
    SELECT date, adj_close
    FROM prices
    WHERE asset_id = 1
    ORDER BY date
""").df()

con.close()

df = df.set_index("date")

daily = compute_returns(df)
monthly = resample_returns(daily, freq="M")

print(daily.dropna().head())
print(monthly.dropna().head())


date
2015-01-02   -0.002647
2015-01-05   -0.010954
2015-01-06   -0.045384
2015-01-07    0.021768
2015-01-08   -0.014339
Name: adj_close, dtype: float64
date
2015-01-31    0.030803
2015-02-28   -0.065228
2015-03-31   -0.034539
2015-04-30    0.044007
2015-05-31    0.029802
Freq: ME, Name: adj_close, dtype: float64


In [20]:
from fincore.storage.connection import get_connection
from fincore.analytics.returns import compute_returns
from fincore.analytics.risk import (
    annualized_volatility,
    sharpe_ratio,
    sortino_ratio,
    value_at_risk,
    conditional_value_at_risk,
)
from fincore.analytics.drawdown import max_drawdown

con = get_connection(read_only=True)

df = con.execute("""
    SELECT date, adj_close
    FROM prices
    WHERE asset_id = 1
    ORDER BY date
""").df()

con.close()

df = df.set_index("date")

returns = compute_returns(df).dropna()

print("Vol:", annualized_volatility(returns))
print("Sharpe:", sharpe_ratio(returns))
print("Sortino:", sortino_ratio(returns))
print("Max DD:", max_drawdown(returns))
print("VaR (5%):", value_at_risk(returns))
print("CVaR (5%):", conditional_value_at_risk(returns))


Vol: 0.2717734772568688
Sharpe: 0.8568793215107972
Sortino: 0.9000763010017451
Max DD: -0.45088360019728735
VaR (5%): -0.023159254829592486
CVaR (5%): -0.03558880470059925


In [21]:
import pandas as pd
from fincore.storage.connection import get_connection
from fincore.analytics.returns import compute_returns
from fincore.portfolio.base import Portfolio
from fincore.portfolio.strategies.equal_weight import EqualWeightStrategy

# Load prices
con = get_connection(read_only=True)
df = con.execute("""
    SELECT date, asset_id, adj_close
    FROM prices
    ORDER BY date
""").df()
con.close()

# Pivot to date x asset
prices = df.pivot(index="date", columns="asset_id", values="adj_close")

# Compute returns
from fincore.analytics.returns import compute_returns_matrix

returns = compute_returns_matrix(prices).dropna()


# Construct portfolio
strategy = EqualWeightStrategy()
portfolio = Portfolio(returns=returns, strategy=strategy)
weights = portfolio.construct()

print(weights)
print("Sum:", weights.sum())


{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.782428Z", "level": "info"}
{"assets": 5, "event": "equal_weight_computed", "timestamp": "2025-12-13T20:55:55.782899Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.783290Z", "level": "info"}
asset_id
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
dtype: float64
Sum: 1.0


In [22]:
from fincore.portfolio.strategies.min_variance import MinimumVarianceStrategy
from fincore.portfolio.base import Portfolio

strategy = MinimumVarianceStrategy()
portfolio = Portfolio(returns=returns, strategy=strategy)
weights = portfolio.construct()

print(weights)
print("Sum:", weights.sum())


{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.790139Z", "level": "info"}
{"assets": 5, "cov_method": "sample", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:55.793166Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.793815Z", "level": "info"}
asset_id
1    0.152619
2    0.310271
3    0.121814
4    0.360508
5    0.054789
dtype: float64
Sum: 1.0


In [23]:
from fincore.portfolio.strategies.risk_parity import RiskParityStrategy
from fincore.portfolio.base import Portfolio

strategy = RiskParityStrategy()
portfolio = Portfolio(returns=returns, strategy=strategy)
weights = portfolio.construct()

print(weights)
print("Sum:", weights.sum())


{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.799722Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.802028Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.802416Z", "level": "info"}
asset_id
1    0.188155
2    0.219998
3    0.193703
4    0.230530
5    0.167614
dtype: float64
Sum: 1.0


In [24]:
from fincore.portfolio.base import Portfolio
from fincore.portfolio.strategies.equal_weight import EqualWeightStrategy
from fincore.portfolio.strategies.min_variance import MinimumVarianceStrategy
from fincore.portfolio.strategies.risk_parity import RiskParityStrategy

# Equal Weight
ew_strategy = EqualWeightStrategy()
ew_portfolio = Portfolio(returns=returns, strategy=ew_strategy)
ew_weights = ew_portfolio.construct()

# Minimum Variance
mv_strategy = MinimumVarianceStrategy()
mv_portfolio = Portfolio(returns=returns, strategy=mv_strategy)
mv_weights = mv_portfolio.construct()

# Risk Parity
rp_strategy = RiskParityStrategy()
rp_portfolio = Portfolio(returns=returns, strategy=rp_strategy)
rp_weights = rp_portfolio.construct()


{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.809209Z", "level": "info"}
{"assets": 5, "event": "equal_weight_computed", "timestamp": "2025-12-13T20:55:55.809711Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.810280Z", "level": "info"}
{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.811152Z", "level": "info"}
{"assets": 5, "cov_method": "sample", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:55.812380Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.812778Z", "level": "info"}
{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:55.813111Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.814331Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:55.814714Z", "

In [25]:
from fincore.portfolio.performance import compute_portfolio_returns
from fincore.analytics.risk import annualized_volatility, sharpe_ratio
from fincore.analytics.drawdown import max_drawdown

# Equal Weight
ew_returns = compute_portfolio_returns(returns, ew_weights)
print("EW Vol:", annualized_volatility(ew_returns))
print("EW Sharpe:", sharpe_ratio(ew_returns))
print("EW Max DD:", max_drawdown(ew_returns))

# Min Variance
mv_returns = compute_portfolio_returns(returns, mv_weights)
print("MV Vol:", annualized_volatility(mv_returns))
print("MV Sharpe:", sharpe_ratio(mv_returns))
print("MV Max DD:", max_drawdown(mv_returns))

# Risk Parity
rp_returns = compute_portfolio_returns(returns, rp_weights)
print("RP Vol:", annualized_volatility(rp_returns))
print("RP Sharpe:", sharpe_ratio(rp_returns))
print("RP Max DD:", max_drawdown(rp_returns))


{"periods": 2704, "event": "portfolio_returns_computed", "timestamp": "2025-12-13T20:55:55.823476Z", "level": "info"}
EW Vol: 0.17704128813974807
EW Sharpe: 1.0132096576383096
EW Max DD: -0.3731435550285539
{"periods": 2704, "event": "portfolio_returns_computed", "timestamp": "2025-12-13T20:55:55.826575Z", "level": "info"}
MV Vol: 0.17040869814355397
MV Sharpe: 0.9923882568923532
MV Max DD: -0.35129754284253906
{"periods": 2704, "event": "portfolio_returns_computed", "timestamp": "2025-12-13T20:55:55.828899Z", "level": "info"}
RP Vol: 0.17468083170475604
RP Sharpe: 1.0144522968450045
RP Max DD: -0.3676580439503861


In [26]:
from fincore.portfolio.backtest import Backtest
from fincore.portfolio.strategies.risk_parity import RiskParityStrategy
from fincore.analytics.risk import annualized_volatility, sharpe_ratio
from fincore.analytics.drawdown import max_drawdown

bt = Backtest(
    returns=returns,
    strategy=RiskParityStrategy(),
    rebalance_freq="M",
    lookback="3Y",
)

rp_bt_returns = bt.run()

print("RP Monthly Vol:", annualized_volatility(rp_bt_returns))
print("RP Monthly Sharpe:", sharpe_ratio(rp_bt_returns))
print("RP Monthly Max DD:", max_drawdown(rp_bt_returns))


{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.844710Z", "level": "info"}


{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.850138Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.857476Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.872089Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.893855Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.897007Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.902584Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.910576Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.913243Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:55.915673Z", "level": "info"}
{"assets": 5, "event": "risk

/Users/yatin/Documents/Investing/fincore/fincore/portfolio/backtest.py:36: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(self.rebalance_freq)
/Users/yatin/Documents/Investing/fincore/fincore/portfolio/backtest.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  lookback_start = rebalance_date - to_offset(self.lookback)


{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.043549Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.045716Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.048437Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.051065Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.054533Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.056562Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.059460Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.061827Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.064680Z", "level": "info"}
{"assets": 5, "event": "risk

In [27]:
bt_q = Backtest(
    returns=returns,
    strategy=RiskParityStrategy(),
    rebalance_freq="Q",
    lookback="5Y",
)

rp_q_returns = bt_q.run()

print("RP Quarterly Vol:", annualized_volatility(rp_q_returns))
print("RP Quarterly Sharpe:", sharpe_ratio(rp_q_returns))
print("RP Quarterly Max DD:", max_drawdown(rp_q_returns))


{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.227607Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.232934Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.237885Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.242731Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.247025Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.251937Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.256430Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.260817Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.265367Z", "level": "info"}
{"assets": 5, "event": "risk

/Users/yatin/Documents/Investing/fincore/fincore/portfolio/backtest.py:36: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  .resample(self.rebalance_freq)


{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.430722Z", "level": "info"}
{"assets": 5, "event": "risk_parity_computed", "timestamp": "2025-12-13T20:55:56.435228Z", "level": "info"}
{"rebalances": 43, "periods": 2645, "rebalance_freq": "Q", "lookback": "5Y", "event": "backtest_complete", "timestamp": "2025-12-13T20:55:56.444266Z", "level": "info"}
RP Quarterly Vol: 0.1735975113527106
RP Quarterly Sharpe: 1.0277108005693276
RP Quarterly Max DD: -0.368468130471099


In [28]:
from fincore.portfolio.turnover import compute_turnover

def average_turnover(weights_history):
    turnovers = []
    prev = None

    for dt, w in weights_history.items():
        if prev is not None:
            turnovers.append(compute_turnover(prev, w))
        prev = w

    return sum(turnovers) / len(turnovers)


print("Monthly Avg Turnover:", average_turnover(bt.weights_history))
print("Quarterly Avg Turnover:", average_turnover(bt_q.weights_history))


Monthly Avg Turnover: 0.004389099985144105
Quarterly Avg Turnover: 0.0067489184070894685


In [29]:
from fincore.portfolio.costs import transaction_cost
from fincore.portfolio.turnover import compute_turnover
from fincore.analytics.risk import annualized_volatility, sharpe_ratio
from fincore.analytics.drawdown import max_drawdown

COST_RATE = 0.001  # 10 bps

net_returns = []
prev_weights = None

for dt, r in rp_bt_returns.items():
    net_r = r

    if dt in bt.weights_history:
        new_weights = bt.weights_history[dt]
        if prev_weights is not None:
            t = compute_turnover(prev_weights, new_weights)
            cost = transaction_cost(t, COST_RATE)
            net_r -= cost
        prev_weights = new_weights

    net_returns.append((dt, net_r))

rp_monthly_net = (
    pd.Series(dict(net_returns))
    .sort_index()
)

print("Monthly NET Vol:", annualized_volatility(rp_monthly_net))
print("Monthly NET Sharpe:", sharpe_ratio(rp_monthly_net))
print("Monthly NET Max DD:", max_drawdown(rp_monthly_net))


Monthly NET Vol: 0.1735618579710273
Monthly NET Sharpe: 1.0352937363434966
Monthly NET Max DD: -0.370925384024851


In [30]:
net_returns_q = []
prev_weights = None

for dt, r in rp_q_returns.items():
    net_r = r

    if dt in bt_q.weights_history:
        new_weights = bt_q.weights_history[dt]
        if prev_weights is not None:
            t = compute_turnover(prev_weights, new_weights)
            cost = transaction_cost(t, COST_RATE)
            net_r -= cost
        prev_weights = new_weights

    net_returns_q.append((dt, net_r))

rp_quarterly_net = (
    pd.Series(dict(net_returns_q))
    .sort_index()
)

print("Quarterly NET Vol:", annualized_volatility(rp_quarterly_net))
print("Quarterly NET Sharpe:", sharpe_ratio(rp_quarterly_net))
print("Quarterly NET Max DD:", max_drawdown(rp_quarterly_net))


Quarterly NET Vol: 0.1735968879271353
Quarterly NET Sharpe: 1.0275589246973003
Quarterly NET Max DD: -0.36847268717925546


In [31]:
from fincore.analytics.robustness import perturb_returns, weight_stability
from fincore.portfolio.strategies.min_variance import MinimumVarianceStrategy

base_strategy = MinimumVarianceStrategy(cov_method="sample")
shrink_strategy = MinimumVarianceStrategy(cov_method="ledoit_wolf")

w_base = base_strategy.compute_weights(returns)

perturbed = perturb_returns(returns)
w_perturbed = base_strategy.compute_weights(perturbed)

print("Sample cov instability:",
      weight_stability(w_base, w_perturbed))

w_shrink = shrink_strategy.compute_weights(returns)
w_shrink_perturbed = shrink_strategy.compute_weights(perturbed)

print("Shrinkage instability:",
      weight_stability(w_shrink, w_shrink_perturbed))


{"assets": 5, "cov_method": "sample", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:56.530992Z", "level": "info"}
{"assets": 5, "cov_method": "sample", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:56.532599Z", "level": "info"}
Sample cov instability: 0.004219703071123189
{"assets": 5, "cov_method": "ledoit_wolf", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:56.535289Z", "level": "info"}
{"assets": 5, "cov_method": "ledoit_wolf", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:56.536489Z", "level": "info"}
Shrinkage instability: 0.004074799023703196


In [32]:
from fincore.portfolio.constraints import apply_constraints
from fincore.portfolio.strategies.min_variance import MinimumVarianceStrategy
from fincore.portfolio.base import Portfolio

strategy = MinimumVarianceStrategy(cov_method="ledoit_wolf")
portfolio = Portfolio(returns=returns, strategy=strategy)

raw_weights = portfolio.construct()

constrained_weights = apply_constraints(
    raw_weights,
    max_weight=0.30,
)

print("Raw weights:")
print(raw_weights)

print("\nConstrained weights:")
print(constrained_weights)

print("\nSum:", constrained_weights.sum())
print("Max weight:", constrained_weights.max())


{"event": "portfolio_construction_start", "timestamp": "2025-12-13T20:55:56.542046Z", "level": "info"}
{"assets": 5, "cov_method": "ledoit_wolf", "event": "min_variance_computed", "timestamp": "2025-12-13T20:55:56.543996Z", "level": "info"}
{"assets": 5, "event": "portfolio_construction_complete", "timestamp": "2025-12-13T20:55:56.544404Z", "level": "info"}
Raw weights:
asset_id
1    0.154935
2    0.305729
3    0.126591
4    0.353236
5    0.059509
dtype: float64

Constrained weights:
asset_id
1    0.164643
2    0.318798
3    0.134524
4    0.318798
5    0.063238
dtype: float64

Sum: 1.0000000000000002
Max weight: 0.31879770960081977


In [33]:
from fincore.data.benchmark_prices import fetch_benchmark_returns

benchmark_returns = fetch_benchmark_returns(
    ticker="^NSEI",
    start="2015-01-01",
)


{"ticker": "^NSEI", "rows": 2695, "event": "benchmark_returns_fetched", "timestamp": "2025-12-13T20:55:57.010293Z", "level": "info"}


In [39]:
benchmark_returns = benchmark_returns["^NSEI"]


In [40]:
from fincore.analytics.benchmark import (
    excess_returns,
    tracking_error,
    information_ratio,
)

excess = excess_returns(
    rp_monthly_net,
    benchmark_returns,
)

print("Active Return:",
      (1 + excess).prod() - 1)

print("Tracking Error:",
      tracking_error(excess))

print("Information Ratio:",
      information_ratio(excess))



Active Return: 0.8042714871001326
Tracking Error: 0.07502424724911329
Information Ratio: 0.7892540717376904


In [41]:
from fincore.analytics.regime.volatility import volatility_regime

vol_signal = volatility_regime(
    rp_monthly_net,
    window=63,
    lookback=756,
)

print(vol_signal)


{'volatility': 0.09986116751652917, 'regime': 'low', 'confidence': 1.527874379059247, 'low_threshold': 0.12226567915270511, 'high_threshold': 0.1460155247129651}


In [42]:
from fincore.analytics.regime.trend import trend_regime

trend_signal = trend_regime(rp_monthly_net)

print(trend_signal)


{'trend_strength': 0.0006554044469482346, 'regime': 'trending', 'confidence': 0.05994533113653292}


In [43]:
from fincore.analytics.regime.correlation import correlation_regime

corr_signal = correlation_regime(
    returns,  # asset returns DataFrame
    lookback=252,
)

print(corr_signal)


{'recent_corr': 0.36736254095238524, 'historical_corr': 0.33346329566136595, 'delta': 0.033899245291019287, 'regime': 'normal', 'confidence': 0.10165810070276574}


In [44]:
from fincore.analytics.regime.breaks import mean_shift_test

break_signal = mean_shift_test(
    rp_monthly_net,
    window=252,
    threshold=1.0,
)

print(break_signal)



{'z_score': -0.10054248764171844, 'delta_mean': -0.0010992681541375407, 'regime': 'stable'}
